In [51]:
pip install langchain langchain-classic

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip show langchain

Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Users/adeb/llm_engineering/llms/lib/python3.11/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate


## Chat API : LangChain


In [6]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [7]:
#You don’t need to pass the key explicitly in the api as an arg unless you want to override it
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm_model="gpt-4o-mini"
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x1793bc6d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x179b19a90>, root_client=<openai.OpenAI object at 0x179222f50>, root_async_client=<openai.AsyncOpenAI object at 0x179b195d0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [20]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [22]:
prompt_template = ChatPromptTemplate.from_template(template_string)


In [23]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [24]:
customer_style = """American English \
in a calm and respectful tone
"""

In [30]:
customer_email = """
Meine Bestellung wurde nicht geliefert!
"""

In [31]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [32]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [33]:
print(customer_messages[0])

content='Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nMeine Bestellung wurde nicht geliefert!\n```\n' additional_kwargs={} response_metadata={}


In [35]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [36]:
print(customer_response.content)

My order has not been delivered.


## Build a pipeline using LCEL
LCEL, or LangChain Expression Language, is a declarative syntax within the LangChain framework for building AI applications, allowing developers to easily chain components like prompts, LLMs, and retrievers using a simple pipe (|) operator to create complex, maintainable workflows

Key Features of LCEL:

1. Declarative Syntax Using Pipe Operators: Chains are constructed by connecting Runnables with the pipe | operator, creating a clear left-to-right data flow.
2. Parallel Execution: Supports execution of independent tasks concurrently using RunnableParallel, reducing end-to-end latency.
3. Guaranteed Asynchronous Support: All chains run asynchronously by default, supporting high-throughput use cases like web servers.
4. Streaming Output: Supports incremental streaming to allow faster time-to-first-token from language models, enhancing responsiveness.
5. Seamless Deployment with LangServe: Chains can be directly deployed in production environments with support for retries, fallbacks and scaling.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Define the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define the prompt
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}?"
)

# 3. Define the chain using LCEL
# The components are "piped" together
chain = prompt | llm | StrOutputParser()

# 4. Invoke the chain
response = chain.invoke({"topic": "a developer"})
print(response)

#note batching which is parallel 
result_with_batch = chain.batch(["dogs", "bartender", "Uber drivers"])
print(result_with_batch)

#note streaming behaviour
for chunk in chain.stream("scientists"):
    print(chunk, flush=True, end="")

Why don't developers trust nature?

Because it has too many bugs!
['Why did the scarecrow adopt a dog?\n\nBecause he needed a "barking" buddy!', 'A man walks into a bar and says to the bartender, "I bet you $100 that I can bite my own eye." The bartender, thinking it\'s an easy win, agrees. The man then takes out his glass eye and bites it. \n\nAfter the bartender hands over the money, the man says, "I bet you another $100 that I can bite my other eye." The bartender thinks surely he can\'t have two glass eyes, so he agrees. The man then takes out his dentures and bites his other eye.', "Why don't Uber drivers play hide and seek?\n\nBecause good luck hiding when your app is always telling you where they are!"]
Why don't scientists trust atoms?

Because they make up everything!

## Memory

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. Define the prompt with a placeholder for history
prompt = ChatPromptTemplate.from_messages([
    ("system", "The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# 2. Build the LCEL chain
chain = prompt | chat

# 3. Setup a session-based history store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 4. Wrap with RunnableWithMessageHistory
# This automatically handles history injection and persistence
conversation_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# 5. Invoke (replaces predict)
config = {"configurable": {"session_id": "demo_session"}}

print(conversation_with_history.invoke({"input": "Hi, my name is Anirban. I work at Akamai."}, config).content)
print(conversation_with_history.invoke({"input": "What is 1+1?"}, config).content)
print(conversation_with_history.invoke({"input": "What is my name?"}, config).content)

Hi Anirban! It’s great to meet you. How’s everything going at Akamai?
1 + 1 equals 2! If you have any more questions, feel free to ask!
Your name is Anirban!


## IGNORE SECTION BELOW

In [5]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

In [6]:
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]


In [17]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain, RouterOutputParser
#from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate # Updated import
from langchain_classic.chains import LLMChain

In [19]:
llm=chat

In [20]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

/var/folders/15/0pjkg1ps3339d77nqc2t_d_m0000gn/T/ipykernel_8001/334181236.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [21]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [22]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [23]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/var/folders/15/0pjkg1ps3339d77nqc2t_d_m0000gn/T/ipykernel_8001/1931158281.py:3: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [24]:
chain.run("What is black body radiation?")
chain.run("what is 2 + 2")
chain.run("Why does every cell in our body contain DNA?")


/var/folders/15/0pjkg1ps3339d77nqc2t_d_m0000gn/T/ipykernel_8001/3462082194.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of physics?'}
> Finished chain.


> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the genetic blueprint for the organism. Here are several key reasons why this is the case:\n\n1. **Genetic Information**: DNA contains the instructions needed for the development, functioning, growth, and reproduction of all living organisms. It encodes the information necessary to produce proteins, which perform a vast array of functions in the body.\n\n2. **Cellular Function**: Each cell type in the body has specific functions, and the DNA in each cell contains the genes that are necessary for that cell's role. For example, muscle cells have genes that help them contract, while nerve cells have genes that support their ability to transmit signals.\n\n3. **Development and Differentiation**: During the development of an organism, all cells originate from a single fertilized egg, which contains DNA. As the organism grows, cells divide and differentiate into various types, but they all retain a complete set of DNA. This ensures t